In [1]:
from datasets import load_dataset
import evaluate
import torch
import os

# Use CPU only
device = torch.device("cpu")
print(f"Using device: {device}")

# Load dataset
dataset = load_dataset("parquet", data_files={
    "train": "data/iamTangsang_dataset/train-00000-of-00001.parquet",
    "validation": "data/iamTangsang_dataset/validation-00000-of-00001.parquet", 
    "test": "data/iamTangsang_dataset/test-00000-of-00001.parquet"
})

# Subset the data
dataset["train"] = dataset["train"].select(range(10000))       
dataset["validation"] = dataset["validation"].select(range(1000))
dataset["test"] = dataset["test"].select(range(200)) 

# Filter out empty strings and ensure data quality
def is_valid_pair(example):
    source = str(example["source"]).strip() if example["source"] is not None else ""
    target = str(example["target"]).strip() if example["target"] is not None else ""
    return len(source) > 0 and len(target) > 0 and len(source) < 1000 and len(target) < 1000

dataset = dataset.filter(is_valid_pair)

print("Dataset sizes after filtering:")
print(f"Train: {len(dataset['train'])}")
print(f"Validation: {len(dataset['validation'])}")
print(f"Test: {len(dataset['test'])}")
print("\nSample:")
print(dataset["train"][0])

/Users/ishanawal/miniforge3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Dataset sizes after filtering:
Train: 10000
Validation: 1000
Test: 200

Sample:
{'source': '"कुनै पनि अन्य सरकारी एजेन्सीले यो जानकारी प्रयोग गर्न सक्दैन, केन्द्रीय सरकार अन्तर्गतका कसैले कुनै पनि हालतमा यो जानकारी पाउँदैनन् र राज्यका अधिकारीहरूमा पनि स्वास्थ्य अधिकारीहरूले मात्र यसलाई प्रयोग गर्न सक्दछन्," उनले भने।', 'target': '"No other government agency can use this information, no one in the commonwealth government at all, and in state authorities, only the health officer can use it.'}


In [2]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small", legacy=False)

def preprocess_function(examples):
    sources = [str(src).strip() for src in examples["source"]]
    targets = [str(tgt).strip() for tgt in examples["target"]]
    inputs = ["translate Nepali to English: " + src for src in sources]
    model_inputs = tokenizer(inputs, max_length=300, truncation=True, padding=True, return_tensors=None)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=300, truncation=True, padding=True, return_tensors=None)

    model_inputs["labels"] = [
        [(token_id if token_id != tokenizer.pad_token_id else -100) for token_id in label]
        for label in labels["input_ids"]
    ]
    
    return model_inputs

In [3]:
print("Tokenizer vocab size:", tokenizer.vocab_size)
print("Pad token id:", tokenizer.pad_token_id)

lengths = []
for i in range(min(1000, len(dataset["train"]))):
    input_text = "translate Nepali to English: " + str(dataset["train"][i]["source"])
    tokens = tokenizer(input_text)["input_ids"]
    lengths.append(len(tokens))

print(f"Max length: {max(lengths)}")
print(f"Average length: {sum(lengths)/len(lengths):.1f}")

Tokenizer vocab size: 250100
Pad token id: 0
Max length: 272
Average length: 30.3


In [4]:
print("Preprocessing datasets...")
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    desc="Tokenizing"
)

tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

sample = tokenized_dataset["train"][0]
for key, value in sample.items():
    if torch.is_tensor(value):
        print(f"{key}: shape {value.shape}, dtype {value.dtype}")
    else:
        print(f"{key}: {type(value)}")


Preprocessing datasets...
input_ids: shape torch.Size([272]), dtype torch.int64
attention_mask: shape torch.Size([272]), dtype torch.int64
labels: shape torch.Size([219]), dtype torch.int64


In [5]:
def find_invalid_tokens(dataset_split, split_name=""):
    invalid_count = 0
    print(f"\nChecking {split_name} set...")
    for i in range(min(100, len(dataset_split))):
        labels = dataset_split[i]["labels"]
        for token_id in labels:
            if token_id != -100 and (token_id >= tokenizer.vocab_size or token_id < 0):
                print(f"❌ Invalid token ID {token_id} at index {i}")
                invalid_count += 1
                break
    print(f"Invalid tokens in {split_name}: {invalid_count}")
    return invalid_count

find_invalid_tokens(tokenized_dataset["train"], "train")
find_invalid_tokens(tokenized_dataset["validation"], "validation")
find_invalid_tokens(tokenized_dataset["test"], "test")


Checking train set...
Invalid tokens in train: 0

Checking validation set...
Invalid tokens in validation: 0

Checking test set...
Invalid tokens in test: 0


0

In [6]:
print("Loading model...")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
model = model.to(device)

Loading model...


In [7]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5-npi-en",
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    generation_max_length=300,
    weight_decay=0.01,
    save_total_limit=6,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=False,
    use_cpu=True,
    dataloader_pin_memory=False,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    gradient_checkpointing=False,
    report_to=None,
)

In [8]:
from transformers import Seq2SeqTrainer

print("Testing data collator...")
test_batch = [tokenized_dataset["train"][i] for i in range(2)]
collated = data_collator(test_batch)
print("Data collator working:")
for key, value in collated.items():
    print(f"  {key}: {value.shape}")

# %%
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


Testing data collator...
Data collator working:
  input_ids: torch.Size([2, 272])
  attention_mask: torch.Size([2, 272])
  labels: torch.Size([2, 219])
  decoder_input_ids: torch.Size([2, 219])


/Users/ishanawal/miniforge3/envs/myenv/lib/python3.11/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [9]:
print("Starting training on CPU...")
trainer.train()
print("Training completed!")

Starting training on CPU...


  0%|          | 100/30000 [02:02<10:35:41,  1.28s/it]

{'loss': 21.2574, 'grad_norm': 44822.11328125, 'learning_rate': 0.00019933333333333334, 'epoch': 0.02}


  1%|          | 200/30000 [04:04<9:53:42,  1.20s/it] 

{'loss': 9.4868, 'grad_norm': 242.44509887695312, 'learning_rate': 0.00019866666666666668, 'epoch': 0.04}


...

{'loss': 1.842, 'grad_norm': 6.313051700592041, 'learning_rate': 6.666666666666667e-07, 'epoch': 5.98}


100%|██████████| 30000/30000 [10:51:08<00:00,  1.13s/it]

{'loss': 1.8363, 'grad_norm': 4.338659286499023, 'learning_rate': 0.0, 'epoch': 6.0}


                                                        
100%|██████████| 30000/30000 [10:52:40<00:00,  1.31s/it]

{'eval_loss': 2.1863021850585938, 'eval_runtime': 89.9491, 'eval_samples_per_second': 11.117, 'eval_steps_per_second': 5.559, 'epoch': 6.0}
{'train_runtime': 39160.9488, 'train_samples_per_second': 1.532, 'train_steps_per_second': 0.766, 'train_loss': 2.524774772644043, 'epoch': 6.0}
Training completed!


In [10]:
trainer.save_model("./mt5-npi-en")
tokenizer.save_pretrained("./mt5-npi-en")

('./mt5-npi-en/tokenizer_config.json',
 './mt5-npi-en/special_tokens_map.json',
 './mt5-npi-en/spiece.model',
 './mt5-npi-en/added_tokens.json')

In [11]:
test_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
print("Test Evaluation:", test_results)


100%|██████████| 100/100 [00:15<00:00,  6.37it/s]

Test Evaluation: {'eval_loss': 2.2883307933807373, 'eval_runtime': 15.8664, 'eval_samples_per_second': 12.605, 'eval_steps_per_second': 6.303, 'epoch': 6.0}


In [13]:
from tqdm import tqdm
import torch

bleu = evaluate.load("bleu")

# For single-GPU CPU/accelerator inference
model.eval()

batch_size = 8  # You can tune this
sources = dataset["test"]["source"]
references = dataset["test"]["target"]

predictions = []

for i in tqdm(range(0, len(sources), batch_size), desc="Generating translations"):
    batch_src = sources[i:i + batch_size]
    batch_inputs = ["translate Nepali to English: " + s for s in batch_src]
    
    inputs = tokenizer(
        batch_inputs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=300
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=300,
            num_beams=4
        )
    
    batch_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions.extend(batch_preds)

# Print a few samples to check translation quality
for i in range(30):
    print(f"\nSource    : {sources[i]}")
    print(f"Reference : {references[i]}")
    print(f"Predicted : {predictions[i]}")

bleu_score = bleu.compute(
    predictions=[p.strip() for p in predictions],
    references=[[r.strip()] for r in references]
)
print("Test BLEU:", bleu_score["bleu"])



Generating translations: 100%|██████████| 25/25 [04:10<00:00, 10.03s/it]


Source    : यसबाट युवकहरु पनि पीडित भएको देखिन्छन ।
Reference : Young people also suffer from it.
Predicted : The young people are not stunned.

Source    : बजाऊने लिस्ट
Reference : Playlist
Predicted : Playing List

Source    : त्यहाँ केही नियमहरू छन् जुन तपाईंले पछ्याउनु पर्छः
Reference : There are a few rules that you should follow:
Predicted : There are some rules that you need to follow:

Source    : सबै गीतहरु उनी आफैंले लेखेका हुन् ।
Reference : All songs were written by himself.
Predicted : He wrote all the songs he wrote.

Source    : मेरो एउटा जर्मन साथी छ ।
Reference : I had a German friend.
Predicted : I have a German friend.

Source    : तपाईँ वास्तवमै फाइल मेट्न चाहनुहुन्छ?

Reference : Do you really want to delete file ?

Predicted : Do you want to delete file?

Source    : यस कुरालाई लिएर आक्रोशित हुनुपर्ने कुनै आवश्यकता छैन ।
Reference : There is no need to get upset about this.
Predicted : There is no need to be worried about this.

Source    : पुरुष र महिला दुबैले य

: 